In [144]:
import tensorflow as tf
from pprint import pprint
from tensorflow.examples.tutorials.mnist import input_data



In [145]:
mnist = input_data.read_data_sets("tmp/data/", one_hot = True)

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
n_classes = 10
batch_size = 100

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [146]:
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

In [147]:
class TfAnn(object):
    
    def __init__(self):
        self.hidden_1_layer = {'weights':[],'biases':[]}
        self.hidden_2_layer = {'weights':[],'biases':[]}
        self.hidden_3_layer = {'weights':[],'biases':[]}
        self.output_layer = {'weights':[],'biases':[]}
        self.np_hidden_1_layer={"weights":[],"biases":[]}
        self.np_hidden_2_layer={"weights":[],"biases":[]}
        self.np_hidden_3_layer={"weights":[],"biases":[]}
        self.np_output_layer={"weights":[],"biases":[]}
    
    def init_empty(self,size,n_nodes_hl1,n_nodes_hl2,n_nodes_hl3,n_classes):
        self.hidden_1_layer = {'weights':tf.Variable(tf.random_normal([size, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

        self.hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

        self.hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

        self.output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                        'biases':tf.Variable(tf.random_normal([n_classes]))}


    def init_values(self,size,l1_weights,l2_weights,l3_weights,out_weights,l1_biases,l2_biases,l3_biases,out_biases):
        self.hidden_1_layer = {'weights':tf.Variable(l1_weights),'biases':tf.Variable(l1_biases)}
        self.hidden_2_layer = {'weights':tf.Variable(l2_weights),'biases':tf.Variable(l2_biases)}
        self.hidden_3_layer = {'weights':tf.Variable(l3_weights),'biases':tf.Variable(l3_biases)}
        self.output_layer = {'weights':tf.Variable(out_weights),'biases':tf.Variable(out_biases)}


    def create(self,data):
        l1 = tf.add(tf.matmul(data,self.hidden_1_layer['weights']), self.hidden_1_layer['biases'])
        l1 = tf.nn.relu(l1)

        l2 = tf.add(tf.matmul(l1,self.hidden_2_layer['weights']), self.hidden_2_layer['biases'])
        l2 = tf.nn.relu(l2)

        l3 = tf.add(tf.matmul(l2,self.hidden_3_layer['weights']), self.hidden_3_layer['biases'])
        l3 = tf.nn.relu(l3)

        output =  tf.add(tf.matmul(l3,self.output_layer['weights']) , self.output_layer['biases'])

        return output
    
    def extract(self):
        self.np_hidden_1_layer["weights"] = neural_network_model.hidden_1_layer["weights"].eval()
        self.np_hidden_2_layer["weights"] = neural_network_model.hidden_2_layer["weights"].eval()
        self.np_hidden_3_layer["weights"] = neural_network_model.hidden_3_layer["weights"].eval()
        self.np_output_layer["weights"] = neural_network_model.output_layer["weights"].eval()
        self.np_hidden_1_layer["biases"] = neural_network_model.hidden_1_layer["biases"].eval()
        self.np_hidden_2_layer["biases"] = neural_network_model.hidden_2_layer["biases"].eval()
        self.np_hidden_3_layer["biases"] = neural_network_model.hidden_3_layer["biases"].eval()
        self.np_output_layer["biases"] = neural_network_model.output_layer["biases"].eval()
        

In [148]:
def train_neural_network(x,neural_network_model):
    prediction = neural_network_model.create(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        neural_network_model.extract()
        

In [150]:
neural_network_model = TfAnn()
neural_network_model.init_empty(784,n_nodes_hl1,n_nodes_hl2,n_nodes_hl3,n_classes)
train_neural_network(x,neural_network_model)
pprint(neural_network_model.np_hidden_1_layer["weights"])

Epoch 0 completed out of 10 loss: 1665077.4106140137
Epoch 1 completed out of 10 loss: 376538.92571258545
Epoch 2 completed out of 10 loss: 207680.11528718472
Epoch 3 completed out of 10 loss: 121175.00202047825
Epoch 4 completed out of 10 loss: 73239.0776475668
Epoch 5 completed out of 10 loss: 50062.821100176574
Epoch 6 completed out of 10 loss: 32304.32735222578
Epoch 7 completed out of 10 loss: 21913.27770452948
Epoch 8 completed out of 10 loss: 18693.822882669418
Epoch 9 completed out of 10 loss: 19557.44637697935
Accuracy: 0.9473
array([[ 0.36754656,  0.00396802, -1.0628358 , ..., -0.55542004,
        -0.4289287 ,  2.0505106 ],
       [-1.4701712 ,  0.19074635,  1.6281519 , ...,  0.57202315,
        -1.7025607 ,  0.9469387 ],
       [ 0.47146574, -1.20716   ,  2.2263017 , ..., -1.9913008 ,
         0.279925  , -0.45598564],
       ...,
       [ 0.7813348 ,  0.03839272,  2.4179583 , ..., -1.0577943 ,
         0.19103912, -1.0859323 ],
       [ 2.2770443 ,  1.3218967 , -0.08537007,

In [153]:
def run_neural_network(x,neural_network_model):
    prediction = neural_network_model.create(x)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        predictions = tf.argmax(prediction, 1)
        correct = tf.equal(predictions, tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        #con = tf.confusion_matrix(labels=y_, predictions=y )
    

In [154]:
test_neural_network_model = TfAnn()
l1_weights = neural_network_model.np_hidden_1_layer["weights"]
l1_biases = neural_network_model.np_hidden_1_layer["biases"]
l2_weights = neural_network_model.np_hidden_2_layer["weights"]
l2_biases = neural_network_model.np_hidden_2_layer["biases"]
l3_weights = neural_network_model.np_hidden_3_layer["weights"]
l3_biases = neural_network_model.np_hidden_3_layer["biases"]
out_weights = neural_network_model.np_output_layer["weights"]
out_biases = neural_network_model.np_output_layer["biases"]

test_neural_network_model.init_values(784,l1_weights,l2_weights,l3_weights,out_weights,l1_biases,l2_biases,l3_biases,out_biases)
run_neural_network(x,test_neural_network_model)


Accuracy: 0.9473
